# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits!

## Game Play

To play a hand of Blackjack the following steps must be followed:

1. Create a deck of 52 cards
2. Shuffle the deck
3. Ask the Player for their bet
4. Make sure that the Player's bet does not exceed their available chips
5. Deal two cards to the Dealer and two cards to the Player
6. Show only one of the Dealer's cards, the other remains hidden
7. Show both of the Player's cards
8. Ask the Player if they wish to Hit, and take another card
9. If the Player's hand doesn't Bust (go over 21), ask if they'd like to Hit again.
10. If a Player Stands, play the Dealer's hand. The dealer will always Hit until the Dealer's value meets or exceeds 17
11. Determine the winner and adjust the Player's chips accordingly
12. Ask the Player if they'd like to play again

## Playing Cards

A standard deck of playing cards has four suits (Hearts, Diamonds, Spades and Clubs) and thirteen ranks (2 through 10, then the face cards Jack, Queen, King and Ace) for a total of 52 cards per deck. Jacks, Queens and Kings all have a rank of 10. Aces have a rank of either 11 or 1 as needed to reach 21 without busting. As a starting point in your program, you may want to assign variables to store a list of suits, ranks, and then use a dictionary to map ranks to values.

# The Game

In [8]:
import random

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight',
         'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6,
          'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10,
          'Queen': 10, 'King': 10, 'Ace': 11}

In [9]:
class Card:

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]

    def __str__(self):
        return self.rank + ' of ' + self.suit

In [11]:
class Deck:

    def __init__(self):
        self.deck = []
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit, rank))
    
    def __str__(self):
        val = ''
        for card in self.deck:
            val += card.__str__() + '\n'
        return val

    def shuffle(self):
        random.shuffle(self.deck)

    def deal(self):
        return self.deck.pop()

In [133]:
class Hand:
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0

    def __str__(self):
        val = ''
        for card in self.cards:
            val += card.__str__() + '\n'
        return val

    def add_card(self, card):
        self.cards.append(card)
        if card.rank == 'Ace':
            self.aces += 1

    def get_score(self):
        score = 0

        for c in self.cards:
            score += c.value
        
        # if we bust with aces, we try to avoid that by switching
        while self.aces > 0 and score > 21:
            score -= 10
            self.aces -= 1
            
        return score
    
    def clear_hand(self):
        self.cards = []

In [141]:
class Player:

    def __init__(self, name):
        self.name = name
        self.total = 100
        self.bet = 0
        self.hand = Hand()
        self.playing = True
        self.bust = False

    def __str__(self):
        return self.name + '\n' + self.hand.__str__() + '\n' + 'Chips: ' + str(self.total) + '\n' + 'Score: ' + str(self.hand.get_score()) + '\n'

    def give_card(self, deck):
        self.hand.add_card(deck.deal())

    def place_bet(self):
        while True:
            try:
                self.bet = int(input(self.name + ", place your bet: "))
            except:
                print("Your bet must be an integer")
                continue
            else:
                if self.bet > self.total:
                    print("You can't bet more than you own")
                    continue
                else:
                    print(self.name + "'s bet: " + str(self.bet) + " chips" + '\n')
                    self.total -= self.bet
                    break
        
    def hit_or_stand(self, deck):
        hit = False
        while True:
            try:
                hit = bool(int(input("Hit (1) or Stand (0): ")))
            except:
                print("You must Hit (1) or Stand(0)")
                continue
            else:
                break

        if hit:
            self.hand.add_card(deck.deal())   
        else:
            print("You stand at: ")
            self.playing = False

        if self.hand.get_score() > 21:
            self.bust = True
            print("You have gone over 21, and have busted\n")
            self.playing = False

    def blackjack(self):
        return len(self.hand.cards) == 2 and self.hand.get_score == 21

    def get_score(self):
        return self.hand.get_score()

    def clear_hand(self):
        self.hand.clear_hand()

    # wining a bet generates different amounts
    # if you hit a blackjack, you get 2.5 the bet
    # if you beat the dealer, its 2
    def win_bet(self, bet_quantifier):
        self.total += self.bet * bet_quantifier

    def lose_bet(self):
        self.total -= self.bet

In [135]:
class Dealer(Player):

    # There's a danger in this method...need to fix on hand side
    def __str__(self):
        if len(self.hand.cards) > 0:
            i_card = self.hand.cards[0]
            return self.name + '\n' + i_card.__str__() + '\n' + 'Score: ' + str(i_card.value) + '\n'
        else:
            return self.name + '\n' + 'No Cards'

    def finishHand(self, deck):
        while self.hand.get_score() < 17:
            self.hit(deck)

    def hit(self, deck):
        self.hand.add_card(deck.deal())   

        if self.hand.get_score() > 21:
            self.bust = True

    def showFullHand(self):
        print(self.name + '\n' + self.hand.__str__() + '\n' + 'Score: ' + str(self.hand.get_score()) + '\n')

Dealer
Seven of Spades
Nine of Clubs
Ace of Hearts

In [145]:
test_deck = Deck()
test_deck.shuffle()

player = Dealer("Test")
player.hand.add_card(Card("Spades", "Seven"))
player.hand.add_card(Card("Clubs", "Nine"))
player.hand.add_card(Card("Hearts", "Ace"))
print(player.get_score())

17


In [136]:
test_deck = Deck()
test_deck.shuffle()

player = Player("Gary")
player.place_bet()

player.give_card(test_deck)
player.give_card(test_deck)

dealer = Dealer("Dealer")
dealer.give_card(test_deck)
dealer.give_card(test_deck)

print(dealer)
print(player)

while player.playing:
    player.hit_or_stand(test_deck)
    print(player)

dealer.finishHand(test_deck)
dealer.showFullHand()

Gary's bet: 50 chips

Dealer
Two of Spades
Score: 2

Gary
Queen of Spades
Ten of Diamonds

Score: 20

You stand at: 
Gary
Queen of Spades
Ten of Diamonds

Score: 20

Dealer
Two of Spades
Eight of Spades
King of Clubs

Score: 20



In [137]:
def handle_payout(player, dealer):
    # Player busts, so they lose
    if player.bust:
        print("You busted, and lost your bet\n")
        player.lose_bet()
        return
    # So, dealer didn't bust and they match the dealer, they 'win' their bet back (push)
    if not dealer.bust and player.get_score() == dealer.get_score():
        print("You match the dealer, push\n")
        player.win_bet(1)
    # Player hit blackjack and dealer either busted or has lower score
    if player.blackjack():
        print("You got a blackjack, 2.5 your bet\n")
        player.win_bet(2.5)
        return
    # Player wins regular. They haven't busted and they haven't hit blackjack
    if dealer.bust:
        print("Dealer busted, 2 your bet\n")
        player.win_bet(2)
        return
    if player.get_score() > dealer.get_score():
        print("You beat the dealer, 2 your bet\n")
        player.win_bet(2)
        return
    if dealer.get_score() > player.get_score():
        print("Dealer beat you, you lost your bet \n")
        player.lose_bet()
        return

In [125]:
def play_next_round():
    while True:
        try:
            playing = int(input("New Round (1) Yes, (0) No: "))
        except:
            print("Bad Input")
            continue
        else:
            if playing > 1 or playing < 0:
                print("(1) Yes, (0) No")
                continue
            else:
                break
        
    return bool(playing)

In [142]:
# Print an opening statement
# I could have a 'choose how many players and their names
print('We are playing blackjack with 4 players')

players = []
players.append(Player("Gary"))
players.append(Player("Katie"))
#players.append(Player("Rowan"))
#players.append(Player("Hazel"))

dealer = Dealer("Dealer")

while True:
    deck = Deck()
    deck.shuffle()

    # Players first place a bet
    for player in players:
        player.place_bet()

    # Go around the table and give a player a card, twice
    for i in range(2):
        for player in players:
            player.give_card(deck)

    # Give dealer 2 cards
    dealer.give_card(test)
    dealer.give_card(test)

    for player in players:
        print(dealer) # Show the players the dealer hand
        print(player) # Show the player their hand

        while player.playing:
            player.hit_or_stand(deck)
            print(player)

    dealer.finishHand(deck)
    
    for player in players:
        dealer.showFullHand()
        print(player)
        handle_payout(player, dealer)
        print(player)

    if not play_next_round():
        break    

We are playing blackjack with 4 players
Gary's bet: 50 chips

Katie's bet: 50 chips

Dealer
Seven of Spades
Score: 7

Gary
Nine of Diamonds
Six of Clubs

Chips: 100
Score: 15

Gary
Nine of Diamonds
Six of Clubs
Five of Diamonds

Chips: 100
Score: 20

You stand at: 
Gary
Nine of Diamonds
Six of Clubs
Five of Diamonds

Chips: 100
Score: 20

Dealer
Seven of Spades
Score: 7

Katie
Queen of Hearts
Four of Clubs

Chips: 100
Score: 14

Katie
Queen of Hearts
Four of Clubs
Five of Spades

Chips: 100
Score: 19

You stand at: 
Katie
Queen of Hearts
Four of Clubs
Five of Spades

Chips: 100
Score: 19

Dealer
Seven of Spades
Nine of Clubs
Ace of Hearts

Score: 27

Gary
Nine of Diamonds
Six of Clubs
Five of Diamonds

Chips: 100
Score: 20

Dealer beat you, you lost your bet 

Gary
Nine of Diamonds
Six of Clubs
Five of Diamonds

Chips: 50
Score: 20

Dealer
Seven of Spades
Nine of Clubs
Ace of Hearts

Score: 27

Katie
Queen of Hearts
Four of Clubs
Five of Spades

Chips: 100
Score: 19

Dealer beat you, y

# Things to Improve

1. Put in option to choose number of players and their names